In [4]:
# from tokenizers import BertWordPieceTokenizer

# tokenizer = BertWordPieceTokenizer("bert-base-uncased-vocab.txt", lowercase=True)

In [39]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())

In [40]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [41]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

In [42]:
import pandas as pd

In [43]:
df = pd.read_excel('clean_test_hugging_face_scientific.xlsx')
df

,article,abstract
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in pa...
1,small non coding rnas are transcribed into mr...,small non coding rnas include sirna mirna ...
2,ohss is a serious complication of ovulation in...,objective to evaluate the efficacy and safet...
3,congenital adrenal hyperplasia cah refers to...,congenital adrenal hyperplasia is a group of ...
4,type 1 diabetes t1d results from the destruc...,objectivespentoxifylline is an immunomodulato...
...,...,...
6653,neurocysticercosis is the most common parasiti...,cysticercosis is an infection with larval cys...
6654,most chd prevalence data are based on populati...,background congenital heart disease chd is...
6655,renal transplantation rates are low among pati...,we describe a successful living donor liver t...
6656,acute kidney injury aki defined as an abrup...,objective the aim was to investigate the eff...


In [44]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [45]:
text = df['article'].tolist()
# tokenizer.train(trainer, files)

In [46]:
len(text)

6658

In [47]:
text.extend(df['abstract'].tolist())

In [48]:
len(text)

13316

In [50]:
from tokenizers import Tokenizer, trainers, normalizers, pre_tokenizers, models,processors  

# Initialize the tokenizer
tokenizer = Tokenizer(models.BPE())

# Set up the tokenizer with necessary settings (e.g., normalizers, pre-tokenizers)
tokenizer.normalizer = normalizers.Sequence([normalizers.Lowercase()])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Train the tokenizer
trainer = trainers.BpeTrainer(vocab_size=10000, min_frequency=2)
tokenizer.train_from_iterator(text, trainer=trainer)

In [52]:
files = tokenizer.model.save("data", "scientific_pubmed")
# tokenizer.model = BPE.from_file(*files, unk_token="[UNK]")

In [54]:
tokenizer.save("data/tokenizer-scientific_pubmed.json")
# tokenizer = Tokenizer.from_file("data/tokenizer-wiki.json")


In [53]:
vocab_size = tokenizer.get_vocab_size()
print(f"The vocabulary size of the trained tokenizer is: {vocab_size}")

The vocabulary size of the trained tokenizer is: 10000


In [59]:
# import trax
# Preprocess your data to prepare for Trax's TFDS structure
# For example, create a list of tuples where each tuple contains your data
prepared_data = [(article, abstract) for article, abstract in zip(train['article'], train['abstract'])]
val_prepared_data = [(article, abstract) for article, abstract in zip(test['article'], test['abstract'])]


def data_generator(data):
    for text, label in data:
        yield text, label

# Create a generator from your prepared data
train_data_stream = data_generator(prepared_data)
val_data_stream = data_generator(val_prepared_data)

In [73]:
# Tokenizing function using Trax Tokenizer
def tokenize_text(text,tokenizer=tokenizer):
    encoded = tokenizer.encode(text)
    return list(encoded.ids)

In [75]:
tokenized_articles = df['article'].apply(tokenize_text)
tokenized_abstracts = df['abstract'].apply(tokenize_text)

# Creating a new DataFrame with tokenized data
df_tokenized = pd.DataFrame({
    'tokenized_article': tokenized_articles,
    'tokenized_abstract': tokenized_abstracts
})

In [79]:
def detokenize_text(token_ids, tokenizer=tokenizer):
    decoded_text = tokenizer.decode(token_ids)
    return decoded_text

In [80]:
detokenized_articles = df_tokenized['tokenized_article'].apply(detokenize_text)
detokenized_abstracts = df_tokenized['tokenized_abstract'].apply(detokenize_text)

# Creating a new DataFrame with tokenized data
df_detokenized = pd.DataFrame({
    'detokenized_article': detokenized_articles,
    'detokenized_abstract': detokenized_abstracts
})

In [81]:
df_detokenized

,detokenized_article,detokenized_abstract
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in par...
1,small non coding rnas are transcri bed into mr...,small non coding rnas include sirna mirna p ir...
2,oh ss is a serious complication of ov ulation ...,objective to evaluate the efficacy and safety ...
3,congenital adrenal hyperplasia ca h refers to ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes t1d results from the destructi...,objectives pen tox ify ll ine is an immuno mod...
...,...,...
6653,neuro cystic er co sis is the most common para...,cystic er co sis is an infection with lar val ...
6654,most chd prevalence data are based on populati...,background congenital heart disease chd is the...
6655,renal transplantation rates are low among pati...,we describe a successful living donor liver tr...
6656,acute kidney injury aki defined as an ab rup t...,objective the aim was to investigate the effic...


In [83]:
# Assuming 'df_tokenized' contains your tokenized sequences

max_length = 0

for index, row in df_tokenized.iterrows():
    article_length = len(row['tokenized_article'])
    abstract_length = len(row['tokenized_abstract'])
    
    max_length = max(max_length, article_length, abstract_length)

print(f"The maximum sequence length in 'article' and 'abstract' columns is: {max_length}")

The maximum sequence length in 'article' and 'abstract' columns is: 7959


In [86]:
count_2048 = 0

for index, row in df_tokenized.iterrows():
    article_length = len(row['tokenized_article'])
    abstract_length = len(row['tokenized_abstract'])
    
    if article_length <= 2048 and abstract_length <= 2048:
        count_2048 += 1

print(f"The number of examples with a sequence length of 2048 is: {count_2048}")

The number of examples with a sequence length of 2048 is: 2431


In [ ]:
### too long of a sequence 

In [115]:
### sentencePiece tokenization
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import pandas as pd
from tokenizers import Tokenizer, trainers, normalizers, pre_tokenizers, models,processors  

from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()
tokenizer.pre_tokenizer = Whitespace()
tokenizer.normalizer = normalizers.Sequence([normalizers.Lowercase()])

tokenizer.train_from_iterator(
    text,
    vocab_size=10_000,
    min_frequency=10,
    show_progress=True,
    limit_alphabet=500,
)

# trainer = trainers.BpeTrainer(vocab_size=10000, min_frequency=2)
# tokenizer.train_from_iterator(text, trainer=trainer)

In [102]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 kB 10.2 MB/s eta 0:00:0000:01


In [88]:
files = tokenizer.model.save("data", "sentencePiece_scientific_pubmed")


In [89]:
tokenizer.save("data/sentencePiece_tokenizer-scientific_pubmed.json")


In [118]:
# Tokenizing function using Trax Tokenizer
def tokenize_text(text,tokenizer=tokenizer):
    encoded = tokenizer.encode(text)
    return list(encoded.ids)

In [119]:
tokenized_articles = df['article'].apply(tokenize_text)
tokenized_abstracts = df['abstract'].apply(tokenize_text)

# Creating a new DataFrame with tokenized data
df2_tokenized = pd.DataFrame({
    'tokenized_article': tokenized_articles,
    'tokenized_abstract': tokenized_abstracts
})

In [120]:
# Assuming 'df_tokenized' contains your tokenized sequences

max_length = 0

for index, row in df2_tokenized.iterrows():
    article_length = len(row['tokenized_article'])
    abstract_length = len(row['tokenized_abstract'])
    
    max_length = max(max_length, article_length, abstract_length)

print(f"The maximum sequence length in 'article' and 'abstract' columns is: {max_length}")

The maximum sequence length in 'article' and 'abstract' columns is: 7959


In [2]:
!pip install tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.0 MB/s eta 0:00:00a 0:00:01
